In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import edward as ed
import numpy as np
import tensorflow as tf

from edward.models import Bernoulli, Normal, Categorical, Dirichlet, Uniform, Multinomial, Empirical
from edward.util import Progbar

Create some sample data:

In [3]:
sample_size = 1000
obs_cond = np.random.choice([0,1,2],[sample_size],p=[.7,.1,.2]) # cold = 0, flu = 1, allergies = 2

runny_nose_cold = np.random.choice([0,1],[sample_size],p=[.1,.9])
headache_cold = np.random.choice([0,1],[sample_size],p=[.5,.5])
fever_cold = np.random.choice([0,1],[sample_size],p=[.99,.01])

runny_nose_flu = np.random.choice([0,1],[sample_size],p=[.5,.5])
headache_flu = np.random.choice([0,1],[sample_size],p=[.5,.5])
fever_flu = np.random.choice([0,1],[sample_size],p=[.5,.5])

runny_nose_al = np.random.choice([0,1],[sample_size],p=[.1,.9])
headache_al = np.random.choice([0,1],[sample_size],p=[.99,.01])
fever_al = np.random.choice([0,1],[sample_size],p=[.99,.01])

runny_nose = np.stack([runny_nose_cold,runny_nose_flu,runny_nose_al])
headache = np.stack([headache_cold,headache_flu,headache_al])
fever = np.stack([fever_cold,fever_flu,fever_al])

In [4]:
obs_runny_nose = np.array([runny_nose[j,i] for i,j in enumerate(obs_cond)])
obs_headache = np.array([headache[j,i] for i,j in enumerate(obs_cond)])
obs_fever = np.array([fever[j,i] for i,j in enumerate(obs_cond)])

In [8]:
cat_p = Normal(tf.constant([.3,.3,.4]),sigma=tf.ones(3))
cond = Categorical(name='cond', logits=tf.ones([sample_size,3])*cat_p)
    
W = Normal(mu=tf.zeros([3,3]),name='W',sigma=tf.ones([3,3]))
    
runny_nose = Bernoulli(name='runny_nose',logits=tf.gather(W[0,:],cond))
headache = Bernoulli(name='headache',logits=tf.gather(W[1,:],cond))
fever = Bernoulli(name='fever',logits=tf.gather(W[2,:],cond))

In [12]:
qcondv = Categorical(name='qcondv', logits=tf.Variable(tf.zeros([sample_size,3],dtype=tf.float32)))
qcat_p = Normal(mu=tf.Variable(tf.zeros(3)),sigma=tf.exp(tf.Variable(tf.zeros(3))))
qW = Normal(mu=tf.Variable(tf.zeros([3, 3])),sigma=tf.exp(tf.Variable(tf.zeros([3, 3]))))

v_inf = ed.KLqp({W:qW,cat_p:qcat_p},
                                {cond:obs_cond,runny_nose:obs_runny_nose,
                                 headache:obs_headache,fever:obs_fever})


In [13]:
v_inf.run()

1000/1000 [100%] ██████████████████████████████ Elapsed: 4s | Loss: 1884.899


In [16]:
np.mean(tf.sigmoid(qW.sample(100)).eval(),axis=0)

array([[ 0.90204674,  0.62354219,  0.86009485],
       [ 0.51929134,  0.50788581,  0.02809707],
       [ 0.01322606,  0.45269677,  0.02887411]], dtype=float32)